# ---
# 
# > ⚠️ *This notebook is a **temporary draft** and will be **removed** after the final Google Colab version is uploaded.*
# 
# ---


# Deep Learning Lab: An Introduction to Multi-agent Artificial Intelligence Systems in Radiology

**Moderator:** Pouria Rouzrokh MD MPH MHPE  
**Speaker:** Moein Shariatnia MD  
**Speaker:** Melina Hosseiny MD

## Overview

In this hands-on workshop, we will explore the concept of multi-agent AI frameworks, emphasizing their importance and unique advantages over single, monolithic models. Attendees will learn how the collaboration of multiple specialized agents can outperform a single powerful model by leveraging task-specific expertise and diverse perspectives. The workshop will provide practical guidance on deploying LLM and VLM agents and connecting them to each other using the Google Agent Development Kit.

## What You'll Learn

This notebook provides a comprehensive introduction to building multi-agent AI systems using Google's Agent Development Kit (ADK). We'll explore:

1. **Basic Agents**: Simple LLM-based agents without tools
2. **Augmented Agents**: Agents enhanced with tools (e.g., Google Search)
3. **Common Pitfalls**: Understanding limitations and failures
4. **Multi-Agent Pipelines**: Sequential, parallel, and loop architectures
5. **Advanced Tools**: Using MCP (Model Context Protocol) servers
6. **Custom Runners**: Building custom execution environments


## Setup and Configuration

Before we begin, we need to install the required packages and configure our environment.


In [ ]:
# Install required packages
%pip install -q google-adk python-dotenv requests pydantic


In [ ]:
# Get API keys from user
import os
from getpass import getpass

print("Please enter your Google API key:")
print("(You can get one from: https://makersuite.google.com/app/apikey)")
api_key = getpass("Google API Key: ")

# Set environment variable
os.environ["GOOGLE_API_KEY"] = api_key

# Optional: Semantic Scholar API key (for networking agent)
print("\nOptional: Enter Semantic Scholar API key (press Enter to skip):")
semantic_key = getpass("Semantic Scholar API Key: ")
if semantic_key:
    os.environ["SEMANTIC_SCHOLAR_API_KEY"] = semantic_key

print("\n✓ Environment configured successfully!")


---

# Part 1: Introduction to an Agent

## What is an Agent?

At its core, an **agent** is a Large Language Model (LLM) enhanced with the ability to perform tasks and make decisions in a semi-automated way. Unlike a plain LLM that only answers a single query, an agent can:

- Receive user input
- Process it using both its pre-trained knowledge and, if available, predefined external tools
- Make decisions step-by-step, continuing its work toward a goal, reaching checkpoints, or halting if guardrails or stopping criteria are triggered
- Return an answer or outcome—not just in response to a question, but potentially after a sequence of actions or automated reasoning steps

A basic agent without any tools behaves just like an LLM (using only its internal knowledge), but more advanced agents can reason, use external tools, and act with degrees of autonomy.

**Key Concept**: Without tools, an agent can only use its training data knowledge. It cannot access real-time information, search the web, or interact with external systems.

Let's explore our first agent: the **Basic Radiology Assistant**.


In [ ]:
# Run the basic radiology assistant agent
# This agent can answer radiology questions but cannot search the web

# Uncomment the line below to run the agent:
# !adk run scripts.agents.basic_radiology_assistant.agent:root_agent


### Understanding the Basic Agent Architecture

```python
from google.adk.agents import Agent

root_agent = Agent(
    name="radiology_assistant",
    model="gemini-2.5-flash",
    instruction="You are a helpful assistant specialized in radiology...",
    tools=[],  # No tools - just pure LLM
)
```

**Takeaway**: A basic agent is a specialized LLM with instructions but no external capabilities.


---

# Part 2: Introduction to Augmenting an Agent with Tools

## Why Add Tools?

While basic agents are useful, they're limited by their training data. To make agents more powerful, we can **augment** them with **tools** that provide:

- Real-time information access (web search)
- External API integration
- Custom function execution
- Database queries

## Google ADK Native Tools

Google ADK provides several built-in tools, including:

- `google_search`: Search the web for real-time information
- Custom function tools: Define your own Python functions
- MCP tools: Integrate with Model Context Protocol servers

Let's explore the **Web Search Agent** that uses Google Search.


In [ ]:
# Run the web search agent
# This agent uses Google Search to answer radiology questions
# Note: In Colab, you'll need to upload the scripts directory first

# Uncomment the line below to run the agent:
# !adk run scripts.agents.web_search_agent.agent:root_agent


### Understanding Tool-Augmented Agents

```python
from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="web_search_agent",
    model="gemini-2.5-flash",
    instruction="You are a radiology specialist that uses web search...",
    tools=[google_search],  # Added tool!
)
```

**Key Concept**: Tools extend an agent's capabilities beyond its training data. The agent can now call `google_search` to find real-time information.


---

# Part 3: A Small Failure - Understanding ADK Limitations

## The Problem: Mixing Different Tool Types

**Important Limitation**: In Google ADK, you **cannot** effectively mix certain types of tools in a single agent. Specifically:

- ❌ **Cannot mix** `google_search` with custom function tools in the same agent
- ❌ **Cannot mix** `google_search` with MCP tools in the same agent
- ❌ **Cannot mix** tools with sub-agents for sequential operations

### Why This Happens

ADK agents are designed to use tools **concurrently**, not **sequentially**. When you need sequential operations (search first, then process results), you need a different architecture.

Let's examine the **Broken Biography Agent** to see this limitation in action.


In [ ]:
# View the broken biography agent code
# This demonstrates the ADK limitation of mixing tool types
# Note: In Colab, you'll need to upload the scripts directory first

# Uncomment to view the broken agent code:
# !cat scripts/agents/biography_agent_broken/agent.py


### The Solution: Multi-Agent Pipelines

When you need sequential operations, you should use **multi-agent pipelines** instead of trying to mix tools in a single agent. This leads us to Part 4!


---

# Part 4: Multi-Agent Pipelines

## What are Multi-Agent Pipelines?

A **multi-agent pipeline** is a system where multiple specialized agents work together, each handling a specific part of a larger task. Google ADK supports three types of pipelines:

### 1. Sequential Pipeline (`SequentialAgent`)

Agents execute **one after another**, passing results from one to the next:

```
Agent 1 → Agent 2 → Agent 3 → Final Result
```

**Use Case**: When each step depends on the previous step's output.

### 2. Parallel Pipeline (`ParallelAgent`)

Agents execute **simultaneously**, and results are combined:

```
         Agent 1 ─┐
                  ├─→ Combined Results
         Agent 2 ─┘
```

**Use Case**: When multiple independent tasks can run at the same time.

### 3. Loop Pipeline (`LoopAgent`)

An agent executes **repeatedly** until a condition is met:

```
Agent → Check Condition → [Loop back if needed] → Final Result
```

**Use Case**: When you need iterative refinement or repeated operations.

## Example: Biography Agent (Sequential Pipeline)

The **working Biography Agent** uses a sequential pipeline:

1. **Research Agent**: Searches for biography information (uses `google_search`)
2. **Writing Agent**: Formats and saves the biography (uses custom function tool)

This solves the problem from Part 3 by separating concerns into different agents!


In [ ]:
# Run the biography agent (sequential pipeline)
# This demonstrates how sequential agents solve the tool mixing problem
# Note: In Colab, you'll need to upload the scripts directory first

# Uncomment the line below to run the agent:
# !adk run scripts.agents.biography_agent.agent:root_agent


### Understanding Sequential Agents

```python
from google.adk.agents import SequentialAgent

root_agent = SequentialAgent(
    name="biography_agent",
    sub_agents=[
        research_agent,  # Step 1: Search
        writing_agent,   # Step 2: Format & Save
    ],
)
```

**Key Concept**: Each agent in the sequence receives the output from the previous agent as input. This allows for sequential operations that single agents cannot handle.


---

# Part 5: Multi-Agent Pipelines with Advanced Tools

## Introduction to MCP (Model Context Protocol)

**MCP (Model Context Protocol)** is a standardized protocol that allows AI agents to interact with external services and tools. Think of it as a way to connect your agents to:

- Web scraping services
- Database systems
- File systems
- Custom APIs
- And much more!

### Why Use MCP?

1. **Standardized Interface**: One protocol for many different services
2. **Reusability**: MCP servers can be shared across projects
3. **Separation of Concerns**: Tools are separate from agent logic
4. **Extensibility**: Easy to add new capabilities

### How MCP Works in ADK

```python
from google.adk.tools.mcp_tool import McpToolset

mcp_tools = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "@modelcontextprotocol/server-fetch"],
        ),
    ),
)

agent = LlmAgent(
    tools=[mcp_tools],  # MCP tools available to agent
)
```

## Example: Radiology Researcher Agent

The **Radiology Researcher Agent** demonstrates:

1. **Sequential Pipeline**: Search → Process
2. **MCP Integration**: Uses Fetch MCP server to retrieve web content
3. **Custom Tools**: Generates CSV files from extracted metadata

This agent finds radiology guidelines and extracts structured metadata.


In [ ]:
# Run the radiology researcher agent (sequential + MCP)
# This demonstrates MCP integration with Fetch server
# Note: In Colab, you'll need to upload the scripts directory first

# Uncomment the line below to run the agent:
# !adk run scripts.agents.radiology_researcher.agent:root_agent


## Example: Networking Agent (Complex Multi-Agent System)

The **Networking Agent** demonstrates a sophisticated multi-agent architecture:

1. **Sequential Pipeline**: Verify → Research → Format
2. **Parallel Pipeline**: URL Finder + Article Search run simultaneously
3. **Multiple Tool Types**: Google Search + Semantic Scholar API + Custom tools
4. **Structured Output**: Returns Pydantic models instead of plain text

This is our most complex example, combining sequential and parallel pipelines!


In [ ]:
# Run the networking agent (complex: sequential + parallel)
# This demonstrates a sophisticated multi-agent system with structured output
# Note: In Colab, you'll need to upload the scripts directory first

# Uncomment the line below to run the agent (replace "Person Name" with actual name):
# !python scripts/run_networking_agent.py "Person Name"


### Understanding Parallel Agents

```python
from google.adk.agents import ParallelAgent

parallel_research = ParallelAgent(
    name="parallel_research_agent",
    sub_agents=[
        url_finder_agent,  # Runs simultaneously
        article_agent,      # Runs simultaneously
    ],
)
```

**Key Concept**: Parallel agents execute at the same time, improving efficiency when tasks are independent. Results are combined before passing to the next sequential step.


---

# Part 6: Running Agents Ourselves - Custom Runners and Sessions

## When to Build Custom Runners?

Sometimes, using the default ADK runner isn't enough. You might want to:

- **Parse structured output** programmatically
- **Customize session management**
- **Add post-processing** steps
- **Integrate with other systems**
- **Control the execution flow** more precisely

### Example: Networking Agent Runner

The networking agent returns a **structured Pydantic model** (JSON-like format). Instead of having the agent create a markdown file directly, we:

1. **Run the agent** using a custom runner
2. **Extract the structured output** (Pydantic model)
3. **Convert to markdown** ourselves
4. **Save the file** programmatically

This gives us more flexibility and control!


## Understanding Runners and Sessions

### What is a Runner?

A **Runner** is responsible for:

- Executing agents
- Managing the execution context
- Handling events and responses
- Coordinating with session services

### What is a Session?

A **Session** tracks:

- Conversation history
- Agent state
- User context
- Message history

**Key Concept**: Custom runners give you fine-grained control over agent execution and output processing. Instead of relying on the agent to format output, you can extract structured data and process it programmatically.


In [ ]:
# Custom Runner Implementation for Networking Agent
# This demonstrates how to build custom runners and sessions
# Note: In Colab, you'll need to upload the scripts directory first

import json
import uuid
import asyncio
from pathlib import Path

# Import the networking agent and its components
from scripts.agents.networking_agent.agent import root_agent
from scripts.agents.networking_agent.sub_agents.formatter_agent import NetworkingProfile
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.runners import types

In [ ]:
def profile_to_markdown(profile: NetworkingProfile) -> str:
    """Convert NetworkingProfile Pydantic model to markdown format."""
    markdown_parts = []
    
    # Header
    markdown_parts.append(f"# {profile.person_name} - Networking Profile\n")
    
    # Background
    markdown_parts.append("## Background\n")
    markdown_parts.append(f"{profile.background}\n")
    
    # Online Presence
    markdown_parts.append("## Online Presence\n")
    if profile.online_presence:
        # Group URLs by category
        categories = {}
        for url_info in profile.online_presence:
            category = url_info.category
            if category not in categories:
                categories[category] = []
            categories[category].append(url_info)
        
        # Write each category
        for category, urls in categories.items():
            markdown_parts.append(f"### {category}\n")
            for url_info in urls:
                markdown_parts.append(f"- [{url_info.platform}]({url_info.url}) - {url_info.description}\n")
    else:
        markdown_parts.append("No online presence information available.\n")
    markdown_parts.append("")
    
    # Recent Publications
    markdown_parts.append("## Recent Publications\n")
    
    # Most Recent Papers
    if profile.recent_publications.most_recent_papers:
        markdown_parts.append("### Most Recent Papers\n")
        for paper in profile.recent_publications.most_recent_papers:
            markdown_parts.append(
                f"- **{paper.title}** - *{paper.journal}* ({paper.year}) "
                f"[Link]({paper.url}) - Citations: {paper.citations}\n"
            )
        markdown_parts.append("")
    
    # Most Cited Papers
    if profile.recent_publications.most_cited_papers:
        markdown_parts.append("### Most Cited Papers\n")
        for paper in profile.recent_publications.most_cited_papers:
            markdown_parts.append(
                f"- **{paper.title}** - *{paper.journal}* ({paper.year}) "
                f"[Link]({paper.url}) - Citations: {paper.citations}\n"
            )
        markdown_parts.append("")
    
    # Contact Information
    if profile.contact_information:
        markdown_parts.append("## Contact Information\n")
        markdown_parts.append(f"{profile.contact_information}\n")
    
    return "".join(markdown_parts)

In [ ]:
async def run_agent_async(person_name: str) -> NetworkingProfile:
    """Run the networking agent with a given person's name (async version)."""
    # Create session service
    session_service = InMemorySessionService()
    
    # Create a unique session ID
    session_id = str(uuid.uuid4())
    user_id = "user"
    app_name = "agents"
    
    # Create the session explicitly before running (as per ADK documentation)
    _ = await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )
    
    # Create runner with the root agent
    runner = Runner(
        agent=root_agent,
        app_name=app_name,
        session_service=session_service
    )
    
    # Create the message content
    message = types.Content(
        parts=[types.Part(text=f"Create a networking profile for {person_name}")],
        role="user"
    )
    
    # Run the agent and collect events using run_async
    events = []
    final_response_text = None
    
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session_id,
            new_message=message
        ):
            events.append(event)
            
            # Check if this is the final response (as per ADK documentation)
            if hasattr(event, 'is_final_response') and event.is_final_response():
                if hasattr(event, 'content') and event.content:
                    if hasattr(event.content, 'parts') and event.content.parts:
                        for part in event.content.parts:
                            if hasattr(part, 'text') and part.text:
                                final_response_text = part.text
                                # Try to parse as JSON (structured output)
                                try:
                                    data = json.loads(part.text)
                                    if isinstance(data, dict) and 'person_name' in data:
                                        return NetworkingProfile(**data)
                                except (json.JSONDecodeError, TypeError, ValueError):
                                    pass
            
            # Also check event.content for structured output
            if hasattr(event, 'content') and event.content:
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            # Try to parse as JSON (structured output)
                            try:
                                data = json.loads(part.text)
                                if isinstance(data, dict) and 'person_name' in data:
                                    return NetworkingProfile(**data)
                            except (json.JSONDecodeError, TypeError, ValueError):
                                pass
    except Exception as e:
        print(f"Error during event processing: {e}")
        import traceback
        traceback.print_exc()
    
    # If we have a final response text but couldn't parse it, try one more time
    if final_response_text:
        try:
            data = json.loads(final_response_text)
            if isinstance(data, dict) and 'person_name' in data:
                return NetworkingProfile(**data)
        except (json.JSONDecodeError, TypeError, ValueError):
            pass
    
    # Debug: Print event types to help diagnose
    if events:
        print(f"Debug: Received {len(events)} events")
        print(f"Debug: Event types: {[type(e).__name__ for e in events[-5:]]}")
        if final_response_text:
            print(f"Debug: Final response text (first 500 chars): {final_response_text[:500]}")
    
    raise ValueError(f"Could not parse NetworkingProfile from agent response. "
                    f"Received {len(events)} events.")


In [ ]:
def run_agent(person_name: str) -> NetworkingProfile:
    """Run the networking agent (synchronous wrapper for async function)."""
    return asyncio.run(run_agent_async(person_name))


print("✓ Custom runner functions loaded!")
print("\nFunctions available:")
print("  - run_agent(person_name): Run the networking agent")
print("  - profile_to_markdown(profile): Convert profile to markdown")
print("\nExample usage:")
print('  profile = run_agent("Pouria Rouzrokh")')
print('  markdown = profile_to_markdown(profile)')
print('  print(markdown)')

In [ ]:
# Example: Run the networking agent and generate a markdown profile
# Uncomment and modify the person_name variable to run

person_name = "..."  # Change this to any radiologist's name

print(f"Researching networking profile for: {person_name}")
print("This may take a few moments...\n")

try:
    # Run the agent
    profile = run_agent(person_name)
    
    # Convert to markdown
    markdown_content = profile_to_markdown(profile)
    
    # Display the markdown (in Colab, this will render nicely)
    from IPython.display import Markdown, display
    display(Markdown(markdown_content))
    
    # Optionally save to file
    # output_path = Path("/content") / f"{person_name.replace(' ', '_').lower()}_profile.md"
    # with open(output_path, 'w', encoding='utf-8') as f:
    #     f.write(markdown_content)
    # print(f"\n✓ Profile saved to: {output_path}")
    
except Exception as e:
    print(f"\n✗ Error running agent: {e}")
    import traceback
    traceback.print_exc()


---

# Part 7: Additional Resources and Advanced Concepts

## Key Concepts We Covered

- ✅ **Basic Agents**: LLM-based agents without tools
- ✅ **Tool-Augmented Agents**: Agents with Google Search and custom tools
- ✅ **ADK Limitations**: Understanding when to use multi-agent pipelines
- ✅ **Sequential Pipelines**: Agents that execute one after another
- ✅ **Parallel Pipelines**: Agents that execute simultaneously
- ✅ **MCP Integration**: Using Model Context Protocol servers
- ✅ **Custom Runners**: Building custom execution environments

## Advanced Concepts to Explore

- Sessions and Session Management
- Callbacks and Event Handling
- Memory and Context Management
- Loop Agents
- Production Deployment

## Official Documentation

📚 **Google ADK Documentation**: https://google.github.io/adk-docs/

📚 **Google Cloud Agent Builder**: https://docs.cloud.google.com/agent-builder/

📚 **Model Context Protocol**: https://modelcontextprotocol.io/

## Summary

In this notebook, we've explored:

1. **Basic agents** - Simple LLM-based assistants
2. **Tool augmentation** - Adding capabilities with tools
3. **Common pitfalls** - Understanding ADK limitations
4. **Multi-agent pipelines** - Sequential, parallel, and loop architectures
5. **MCP integration** - Using external services via Model Context Protocol
6. **Custom runners** - Building custom execution environments

You now have a solid foundation for building sophisticated multi-agent AI systems!

## Next Steps

- Experiment with the agents in this repository
- Try building your own multi-agent system
- Explore MCP servers for your use case
- Read the official ADK documentation
- Join the ADK community for support and updates
